# Задача

Международный сервис по продаже билетов решил подвести итоги театрального сезона. В качестве одной из метрик руководитель проекта хочет посчитать количество пользователей, которые покупали билеты на разные спектакли.

При покупке билета пользователь указывает номер своего телефона. Необходимо найти спектакль с наибольшим числом уникальных телефонных номеров. И посчитать количество соответствующих уникальных телефонных номеров.

In [10]:
import pandas as pd
import numpy as np
from tkinter import filedialog as fd
from tkinter import Tk, BOTH
from datetime import datetime
from datetime import timedelta
import re

regex = "[a-zA-Zа-яА-ЯёЁ]+$"

pattern = re.compile(regex)

import warnings
warnings.filterwarnings("ignore")

## Импортируем данные

In [87]:
root = Tk()

ticket = fd.askopenfilename()

ticket_data = pd.read_csv(ticket, sep=',', encoding='utf-8', header=None)

root.destroy()

## Добавляем названия столбцов

In [88]:
ticket_data.columns = ['Спектакль', 'Номер_телефона']

## Очищаем номера от лишних символов, в том числе букв

In [89]:
incorrect_numbers = []

letters_to_number_dict = {
    'a': 2, 'b': 2, 'c': 2, 'd': 3, 'e': 3, 'f': 3, 'g': 4, 'h': 4, 'i': 4, 'j': 5, 'k': 5, 'l': 5, 'm': 6, 'n': 6,
    'o': 6, 'p': 7, 'q': 7, 'r': 7, 's': 7, 't': 8, 'u': 8, 'v': 8, 'w': 9, 'x': 9, 'y': 9, 'z': 9}

def replace_letter_to_number(char):
    """
    Функция проверяет символ и в случае если символ это буква, 
    возвращает соответствующую цифру из словаря letters_to_number_dict
    """
    if char.isnumeric():
        return char
    elif pattern.search(char):
        return letters_to_number_dict[char]
    else:
        return char
            


def fix_phone_number(phone):
    """
    Функция сначала проверяет на наличие букв в телефонном номере (по формату США),
    если находит буквы, то заменяет их цифрами из словаря letters_to_number_dict.
    После этого извлекает из строки с номером только цифры, проверяет длину 
    и возвращает корректный номер телефона без первой 8 или 7 (если есть 7).
    """
    
    if pattern.search(phone) is not None:
        s_1 = [replace_letter_to_number(letter) for letter in phone]
        phone = ''.join([str(s) for s in s_1])

    phone = ''.join(filter(str.isdigit, phone))
    if len(phone) < 10:
        incorrect_numbers.append(phone)
        return
    return phone[1:]

In [90]:
new_ticket_data = ticket_data.copy()

In [92]:
new_ticket_data['Отформатированный_номер_телефона'] = new_ticket_data['Номер_телефона'].apply(fix_phone_number)

In [93]:
new_ticket_data

,Спектакль,Номер_телефона,Отформатированный_номер_телефона
0,Кошки,+88029264576,8029264576
1,Норма,8805-630-71-19,8056307119
2,Евгений Онегин,8-(802)-235-30-28,8022353028
3,Вестсайдская история,8-803-151-78-95,8031517895
4,Дон Кихот,8-802-311-48-72,8023114872
...,...,...,...
3518,Кармен,8805-642-18-91,8056421891
3519,Баядерка,8-802-618-97-14,8026189714
3520,Кармен,+88034956301,8034956301
3521,Вестсайдская история,+88078994876,8078994876


## Находим спектакль с наибольшим числом уникальных телефонных номеров и считаем их количество

In [105]:
top_5_perf_with_uniq_phone = new_ticket_data\
                             .groupby('Спектакль')['Отформатированный_номер_телефона']\
                             .nunique()\
                             .sort_values(ascending=False)\
                             .to_frame()\
                             .head(5)

top_5_perf_with_uniq_phone

,Отформатированный_номер_телефона
Спектакль,
Кошки,180
Спящая красавица,175
Дон Кихот,175
Кармен,171
Лебединое озеро,171
